In [1]:
#! python2
# -*- coding: utf-8 -*-
import pprint
import pymongo
import datetime
import numpy as np
from pymongo import MongoClient

from scipy import spatial

import pylab

import os

import progressbar

db_name = 'twitter'
col_name = 'after_process'

In [2]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

from igraph import *
import igraph
import math
from operator import itemgetter



In [3]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[col_name].find({})

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    for doc in cursor:
        data += 1
        bar.update(data)
        
        ts = long(doc['timestamp'])
        db['replab'].update({'_id':doc['_id']},{"$set":{'ts':ts}}, upsert=True)
    bar.finish()
#     print len(date)

KeyError: 'timestamp'

In [ ]:
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, unicode):
            return (object.encode('thai'), True, False)
        return pprint.PrettyPrinter.format(self, object, context, maxlevels, level)

def go_print( input ):
    MyPrettyPrinter().pprint(input)
    # ppp = pprint.PrettyPrinter(indent=4)
    # ppp.pprint(input)
    return;

def get_midnight(time):
    return time.replace(minute=0, hour=0, second=0, microsecond=0)

def get_thai_midnight(time):
    out = time + datetime.timedelta(hours=7)
    out = out.replace(minute=0, hour=0, second=0, microsecond=0) - datetime.timedelta(hours=7)
    return out

In [ ]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[col_name].find({}).sort([('ts', pymongo.ASCENDING)])
    

    date = []
    text = []
    text_2 = []
    all_doc = []

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    for doc in cursor:
        data += 1
        bar.update(data)
        
        ts = doc['ts']
        datetime_object = datetime.datetime.fromtimestamp(ts/1000)
        if True:
#         if len(doc['hashtags']) != 0:
            buf = doc['nouns']
            buf_2 = doc['nouns_nltk']
            for x in doc['hashtags']:
                if not x.lower() in buf:
                    buf.append(x.lower())
                if not x.lower() in buf_2:
                    buf_2.append(x.lower())
            all_doc.append(doc)
            date.append(datetime_object)
            text.append(' '.join(buf))
            text_2.append(' '.join(buf_2))
    bar.finish()
#     print len(date)

In [ ]:
def two_str(num):
    if num < 10:
        return str(0)+str(num)
    else:
        return str(num)

In [ ]:
#time_gap <= 60
def split_time_gap(date,text,all_doc,time_gap):
    s_text = {}
    s_all_doc = {}
    for datetime_object in date:
        i = date.index(datetime_object)
        m = datetime_object.minute
        h = datetime_object.hour
        d = datetime_object.day
        key = two_str(d)+'/'+two_str(datetime_object.month)+'::'+two_str(h)+'.'+two_str((m/time_gap)*time_gap)
        if not s_text.has_key(key):
            s_text[key] = []
            s_all_doc[key] = []
        s_text[key].append(text[i])
        s_all_doc[key].append(all_doc[i])
    return s_text,s_all_doc
s_text,s_all_doc = split_time_gap(date,text,all_doc,60)
s_text_2,s_all_doc = split_time_gap(date,text_2,all_doc,60)

In [ ]:
from math import *
import operator as op
import itertools

L = dict() # Lattice
Lkey = dict() # Key of each itemset
X = 0
N = 0

for c in range(100):
    L[c] = dict()
    Lkey[c] = dict()

In [ ]:
#  n เลือก r ไม่สนลำดับ
def nCr(n, r):
    if n < r: return 0
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, range(n, n-r, -1))
    denom = reduce(op.mul, range(1, r+1))
    return numer//denom

In [ ]:
def streamMining(stream, theta, epsilon):
    global L, Lkey
    for c in range(100):
        L[c] = dict()
        Lkey[c] = dict()
    X = 0
    N = 0
    print("Run streamMining")
    T = list() # Buffer
    f = 0 # Average 2-itemsets per transaction
    c = 0 # Number of call -> reducFreq
    for t in stream:
        T.append(t)
        update(t, 1)
        update(t, 2)
        f = twoItemsetPerTransaction(t)
        if len(L[2]) >= ceil(1/(theta*epsilon)) * f:
            reducFreq(2)
            c += 1
            i = 2
            while len(L[i]) != 0:
                i += 1
                for tp in T:
                    if len(tp) >= i:
                        update(tp, i)
                reducFreq(i)
            for ii in range(i,100):
                L[ii] = dict()
                Lkey[ii] = dict()
            T = list()
    for k,v in L.items():
        sorted_v = sorted(v.items(), key=operator.itemgetter(1), reverse=True)
        L[k] = sorted_v
    return L

In [ ]:
def twoItemsetPerTransaction(t):
    global X, N
    f = 0
    N += 1
    X += nCr(len(t), 2)
    f = ceil(X/N)
    return f

In [ ]:
def update(t, i):
    global L, Lkey
    itemsets = createItemsets(t, i)

    for s in itemsets:
        if len(set(s)) != i:
            continue
        if s in L[i]:
            L[i][s] += 1
        elif i <= 2:
            L[i][s] = 1
        elif i > 2:
            mini_itemsets = createItemsets(s, i-1)
            chk = True
            for mini in mini_itemsets:
                if mini not in L[i-1]:
                    chk = False
                    break
            if chk:
                L[i][s] = 1

In [ ]:
def reducFreq(i):
    global L
    remove = list()
    for s in L[i]:
        L[i][s] -= 1
        if L[i][s] <= 0:
            remove.append(s)
    for r in remove:
        del L[i][r]

In [ ]:
def createItemsets(t, i):
    return tuple(map( lambda w: tuple(sorted(w)), list(itertools.combinations(t, i)) ) )

In [ ]:
output= {}
for k,v in s_text.items():
    sample = v
    sample_doc = s_all_doc[k]

    x = streamMining(map( lambda w: w.split(' '), sample ),0.01,0.01)
    
    output[k] = x.copy()

In [ ]:
#     output_text 
#     output_nouns 
#     out_nmf 
#     out_lda 
for k,v in sorted(output.items()):
    print '------' ,k, '-----'
    try:
        
        for data in v.values():
            remain = 15
            for top in data:
                print top
                remain -= 1
                if remain == 0:
                    break
    except :
        pass
    

In [ ]:
output_2= {}
for k,v in s_text_2.items():
    sample = v
    sample_doc = s_all_doc[k]

    x = streamMining(map( lambda w: w.split(' '), sample ),0.01,0.01)
    
    output_2[k] = x.copy()

In [ ]:
#     output_text 
#     output_nouns 
#     out_nmf 
#     out_lda 
for k,v in sorted(output_2.items()):
    print '------' ,k, '-----'
    try:
        for data in v.values():
            remain = 15
            for top in data:
                print top
                remain -= 1
                if remain == 0:
                    break
    except :
        pass
    